# Rainbow Trading Agent

Pytorch implementation based on original Tensorflow implementation from Clement Perroud

1. https://github.com/ClementPerroud/RL-Trading-Agent
2. https://github.com/ClementPerroud/Rainbow-Agent

In [1]:
from lab.utils import (
    add_features,
    reward_function,
    max_drawdown,
    make_env,
    make_agent,
    train,
    evaluation,
    plot_agent,
    dump_agent,
)

## Create Environments

In [2]:
import gymnasium as gym
import gym_trading_env

training_envs = gym.vector.SyncVectorEnv([lambda: make_env("data/processed/training/*.pkl") for _ in range(5)])
validation_envs = gym.vector.SyncVectorEnv([lambda: make_env("data/processed/validation/*.pkl") for _ in range(5)])

Using  directory: data/processed/training/*.pkl
Using dataset directory: /Users/arcmode/code/trading-rl/data/processed/training/*.pkl
Using  directory: data/processed/training/*.pkl
Using dataset directory: /Users/arcmode/code/trading-rl/data/processed/training/*.pkl
Using  directory: data/processed/training/*.pkl
Using dataset directory: /Users/arcmode/code/trading-rl/data/processed/training/*.pkl
Using  directory: data/processed/training/*.pkl
Using dataset directory: /Users/arcmode/code/trading-rl/data/processed/training/*.pkl
Using  directory: data/processed/training/*.pkl
Using dataset directory: /Users/arcmode/code/trading-rl/data/processed/training/*.pkl
Using  directory: data/processed/validation/*.pkl
Using dataset directory: /Users/arcmode/code/trading-rl/data/processed/validation/*.pkl
Using  directory: data/processed/validation/*.pkl
Using dataset directory: /Users/arcmode/code/trading-rl/data/processed/validation/*.pkl
Using  directory: data/processed/validation/*.pkl
Usin

## Initialize Agent

In [3]:
agent = make_agent()

## Training

In [ ]:
NUM_STEPS = 30_000
while True:
    train(agent, training_envs, validation_envs, steps = NUM_STEPS)
    # evaluation(agent, validation_envs)

___________________________________________ TRAINING ___________________________________________
___________________________________________ VALIDATION ___________________________________________
___________________________________________ VALIDATION ___________________________________________
___________________________________________ VALIDATION ___________________________________________
___________________________________________ VALIDATION ___________________________________________
___________________________________________ VALIDATION ___________________________________________
___________________________________________ TRAINING ___________________________________________
___________________________________________ VALIDATION ___________________________________________
___________________________________________ VALIDATION ___________________________________________
___________________________________________ VALIDATION ___________________________________________
______________

In [ ]:
plot_agent()

In [ ]:
# dump_agent()